# STL Ornament Generation Demo

This notebook demonstrates how to create 3D-printable STL files from complex function visualizations.

## Workflow Overview
1. Visualize the complex function in 2D
2. Preview the 3D Riemann sphere
3. Generate STL files for 3D printing
4. Validate the mesh quality

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import complexplorer as cp
from complexplorer.plots_3d_pyvista import riemann_pv
from complexplorer.stl_export import OrnamentGenerator
import pyvista as pv


# IMPORTANT: If you get shader errors, use static backend instead of trame
# Uncomment the line below to switch to static images (no shader issues)
# pv.set_jupyter_backend('static')

# Default is trame for interactive 3D
pv.set_jupyter_backend('trame')
print(f"PyVista version: {pv.__version__}")
print(f"Backend: {pv.global_theme.jupyter_backend}")
print("\nIf you see black outputs or shader errors, change backend to 'static' above")

## 1. Define and Visualize a Complex Function

Let's start with a beautiful rational function:

In [ ]:
# Define the complex function
def f(z):
    return (z**2 - 1) / (z**2 + 1)

def f(z):
    return z

# Create a domain for visualization
domain = cp.Rectangle(real=4, imag=4)

# Choose a colormap
cmap = cp.Phase(n_phi=12, auto_scale_r=True)

# Visualize the function
plt.figure(figsize=(10, 5))

# Plot domain and codomain side by side
cp.pair_plot(domain, f, cmap=cmap)
plt.tight_layout()
plt.show()

## 2. Preview the 3D Riemann Sphere

Before generating STL files, let's see how the function looks on the Riemann sphere:

In [ ]:
# Create a 3D landscape view
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), subplot_kw={'projection': '3d'})

# Analytic landscape
cp.plot_landscape(domain, func=f, cmap=cmap, ax=ax1, z_max=5)
ax1.set_title('Analytic Landscape')

# Riemann sphere (matplotlib version for quick preview)
cp.riemann(f, n=200, cmap=cmap, ax=ax2)
ax2.set_title('Riemann Sphere')

plt.tight_layout()
plt.show()

In [ ]:
riemann_pv(
    func=f,
    n_theta=400,
    n_phi=400,
    scaling='arctan', 
    #scaling_params={'r_min': 0.2, 'r_max': 1.0},
    anti_aliasing=True,
    high_quality=True,
    title="Möbius Transformation: f(z) = (z-1)/(z+1)"
)

## 3. Generate STL Files

Now let's create STL files for 3D printing. The new approach creates complete watertight meshes - you can cut them at any angle in your slicer!

### Example 1: Arctan Scaling (Smooth transitions)

In [ ]:
# Create ornament generator with arctan scaling
ornament1 = OrnamentGenerator(
    func=f,
    resolution=200,  # Medium resolution for balance of quality and speed
    scaling='arctan',
    scaling_params={'r_min': 0.2, 'r_max': 0.95},
    cmap=cmap
)

# Generate the complete ornament mesh
print("Generating ornament with arctan scaling...")
stl_file1 = ornament1.generate_ornament(
    output_file='demo_arctan.stl',
    size_mm=70,       # 70mm diameter
    smooth=True,
    smooth_iterations=25,
    verbose=True
)

print(f"\nGenerated file: {stl_file1}")
print("You can now import this into your slicer and cut at any angle!")

### Example 2: Linear Clamp Scaling (Sharp features)

In [ ]:
# Different scaling for comparison
ornament2 = OrnamentGenerator(
    func=f,
    resolution=150,
    scaling='linear_clamp',
    scaling_params={'m_max': 5, 'r_min': 0.3, 'r_max': 1.0},
    cmap=cmap
)

print("Generating ornament with linear clamp scaling...")
stl_file2 = ornament2.generate_ornament(
    output_file='demo_linear.stl',
    size_mm=70,
    smooth=True,
    smooth_iterations=25,
    verbose=False  # Less verbose this time
)

print(f"\nGenerated file: {stl_file2}")

## 4. Validate Mesh Quality

Let's check the quality of our generated meshes:

In [ ]:
# Load and analyze one of the generated STL files
mesh = pv.read(stl_file1)

print("Mesh Statistics:")
print(f"  Vertices: {mesh.n_points:,}")
print(f"  Triangles: {mesh.n_cells:,}")
print(f"  File size: {mesh.n_points * 3 * 4 / 1024 / 1024:.2f} MB (approx)")

# Check bounds
bounds = mesh.bounds
print(f"\nDimensions (mm):")
print(f"  X: {bounds[1] - bounds[0]:.1f}")
print(f"  Y: {bounds[3] - bounds[2]:.1f}")
print(f"  Z: {bounds[5] - bounds[4]:.1f}")

# Check if watertight
edges = mesh.extract_feature_edges(boundary_edges=True)
print(f"\nWatertight: {'Yes' if edges.n_cells == 0 else 'No ('+str(edges.n_cells)+' boundary edges)'}")

# The mesh is a complete sphere - no flat bottom until you cut it in your slicer!
print("\nThis is a complete sphere mesh.")
print("Import into your slicer to cut at any angle you prefer.")

## 5. Visualize the Generated Mesh

Let's see the complete mesh in 3D:

In [ ]:
# Note: This may not display properly in Jupyter due to backend issues
# Run the visualization examples from the command line for best results

# Load the complete mesh
mesh = pv.read(stl_file1)

# Create a simple plot
plotter = pv.Plotter()
plotter.add_mesh(mesh, color='lightblue', show_edges=True, edge_color='gray')
plotter.add_text('Complete Ornament Mesh', position='upper_left')
plotter.view_isometric()
plotter.show()

print("\nThis is the complete mesh. In your slicer you can:")
print("- Cut at any angle (not just along axes)")
print("- Make hollow with vase mode or infill settings")
print("- Add supports if needed for your cut orientation")

## 6. Try Different Functions

Here are some interesting functions to experiment with:

In [ ]:
# Function gallery
functions = {
    "Möbius": lambda z: (2*z + 1) / (z - 1j),
    "Cubic": lambda z: z**3 - 1,
    "Sine": lambda z: np.sin(z),
    "Rational": lambda z: (z - 0.5) * (z + 0.5) / (z**2 + 0.25),
    "Exponential": lambda z: np.exp(z/2)
}

# Quick visualization of all functions
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()

for idx, (name, func) in enumerate(functions.items()):
    if idx < 6:
        cp.plot(domain, func, cmap=cp.Phase(12), ax=axes[idx])
        axes[idx].set_title(name)
        axes[idx].set_xlabel('')
        axes[idx].set_ylabel('')

# Hide the last subplot if we have less than 6 functions
if len(functions) < 6:
    axes[-1].axis('off')

plt.tight_layout()
plt.show()

print("To generate STL for any of these, just replace the function in the OrnamentGenerator!")

## 7. Tips for 3D Printing

### Slicer Settings
- **Layer Height**: 0.15-0.2mm for good quality
- **Infill**: 20-30% is usually sufficient
- **Supports**: Not needed! The flat bottom ensures good adhesion
- **Print Speed**: Standard speeds work well

### Assembly
1. Print both halves separately
2. Clean any stringing or artifacts
3. Use cyanoacrylate glue or epoxy to join halves
4. Optional: Add a loop or hook for hanging

### Function Selection
- Functions with poles create interesting "spikes" or "valleys"
- Periodic functions (sin, cos) create wave patterns
- Polynomials often have nice symmetry
- Rational functions can combine multiple features

### Troubleshooting
- **Gaps in slicer**: Increase resolution or smoothing iterations
- **Too large**: Reduce size_mm parameter
- **Lost details**: Try different scaling methods
- **Spikes**: Enable spike removal for high-frequency functions

## 8. Advanced: Custom Color Patterns

While STL files don't store color, different patterns create interesting textures:

In [ ]:
# Example with different colormaps that affect the mesh structure
colormaps = [
    ("Phase Portrait", cp.Phase(n_phi=6)),
    ("Chessboard", cp.Chessboard(spacing=0.5)),
    ("Polar Chessboard", cp.PolarChessboard(n_phi=8, spacing=0.3)),
    ("Log Rings", cp.LogRings(base=2))
]

# Visualize how different colormaps look
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = axes.flatten()

test_func = lambda z: (z**2 - 1) / (z**2 + 1)

for idx, (name, cmap) in enumerate(colormaps):
    cp.plot(domain, test_func, cmap=cmap, ax=axes[idx])
    axes[idx].set_title(name)

plt.tight_layout()
plt.show()

print("Note: These patterns affect visualization but not the 3D shape.")
print("The shape is determined by the function's modulus and the scaling method.")

## Clean Up

Remove the demo files if you don't need them:

In [ ]:
# Uncomment to remove generated files
# import os
# for file in [stl_file1, stl_file2]:
#     if os.path.exists(file):
#         os.remove(file)
#         print(f"Removed {file}")